# Whisper v3 is here!

Whisper v3 is a new model open sourced by OpenAI. The model can do multilingual transcriptions and is quite impressive. For example, you can change from English to Spanish or Chinese in the middle of a sentence and it will work well!

The model can be run in a free Google Colab instance and is integrated into `transformers` already, so switching can be a very smooth process if you already use the previous versions.

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/transformers gradio

Let's use the high level `pipeline` from the `transformers` library to load the model.

In [ ]:
import torch
from transformers import pipeline, MarianMTModel, MarianTokenizer

pipe = pipeline("automatic-speech-recognition",
               "openai/whisper-large-v3",
               torch_dtype=torch.float16,
               device="cuda:0")

model_name_translate = "Helsinki-NLP/opus-mt-en-ar"
tokenizer_translation = MarianTokenizer.from_pretrained(model_name_translate)
model_translate = MarianMTModel.from_pretrained(model_name_translate)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
pipe("https://cdn-media.huggingface.co/speech_samples/sample1.flac")

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


{'text': " going along slushy country roads and speaking to damp audiences in draughty schoolrooms day after day for a fortnight he'll have to put in an appearance at some place of worship on sunday morning and he can come to us immediately afterwards"}

Let's now build a quick Gradio demo where we can play with the model directly using our microphone! You can run this code in a Google Colab instance (or locally!) or just head to the <a href="https://huggingface.co/spaces/hf-audio/whisper-large-v3" target="_blank">Space</a> to play directly with it online.

In [ ]:
import gradio as gr

In [ ]:
# import gradio as gr

# def translate(sentence):
#     batch = tokenizer_translation([sentence], return_tensors="pt")
#     generated_ids = model_translate.generate(batch["input_ids"])
#     text  = tokenizer_translation.batch_decode(generated_ids, skip_special_tokens=True)[0]
#     return text

# def transcribe(inputs):
#     if inputs is None:
#         raise gr.Error("No audio file submitted! Please record an audio before submitting your request.")

#     text = pipe(inputs, generate_kwargs={"task": "transcribe"}, return_timestamps=True)["text"]
#     print(text)
#     print(type(text))
#     text = translate(text)
#     return text

# demo = gr.Interface(
#     fn=transcribe,
#     inputs=[
#         gr.Audio(sources=["microphone", "upload"], type="filepath"),
#     ],
#     outputs="text",
#     title="Whisper Large V3: Transcribe Audio",
#     description=(
#         "Transcribe long-form microphone or audio inputs with the click of a button! Demo uses the"
#         " checkpoint [openai/whisper-large-v3](https://huggingface.co/openai/whisper-large-v3) and 🤗 Transformers to transcribe audio files"
#         " of arbitrary length."
#     ),
#     allow_flagging="never",
# )

# demo.launch()


In [ ]:
def translate(sentence):
    batch = tokenizer_translation([sentence], return_tensors="pt")
    generated_ids = model_translate.generate(batch["input_ids"])
    text  = tokenizer_translation.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return text

def transcribe(inputs):
    if inputs is None:
        raise gr.Error("No audio file submitted! Please record an audio before submitting your request.")

    text = pipe(inputs, generate_kwargs={"task": "transcribe"}, return_timestamps=True)["text"]
    print(text)
    print(type(text))
    text = translate(text)
    return text

demo = gr.Blocks(theme=gr.themes.Ocean())

mf_transcribe = gr.Interface(
    fn=transcribe,
    inputs=[
        gr.Audio(sources="microphone", type="filepath"),
    ],
    outputs="text",
    title="Real-Time Speech Translation From English to Arabic",
    description=(
        "Real Time Speech Translation Model from English to Arabic. This model uses the Whisper For speech to generation"
        "then Helensiki model fine tuned on a translation dataset for translation"
    ),
    allow_flagging="never",
)

file_transcribe = gr.Interface(
    fn=transcribe,
    inputs=[
        gr.Audio(sources="upload", type="filepath", label="Audio file"),
    ],
    outputs="text",
    title="Real-Time Speech Translation From English to Arabic",
    description=(
        "Real Time Speech Translation Model from English to Arabic. This model uses the Whisper For speech to generation"
        "then Helensiki model fine tuned on a translation dataset for translation"
    ),
    allow_flagging="never",
)


with demo:
    gr.TabbedInterface([mf_transcribe, file_transcribe], ["Microphone", "Audio file"])

demo.queue().launch(ssr_mode=False)

/usr/local/lib/python3.10/dist-packages/gradio/interface.py:393: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b2ee7818e55ee6d28f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
